In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE182797"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE182797"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE182797.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE182797.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE182797.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic profiling of adult-onset asthma related to damp and moldy buildings and idiopathic environmental intolerance [nasal biopsy]"
!Series_summary	"The objective of the study was to characterize distinct endotypes of asthma related to damp and moldy buildings and to evaluate the potential molecular similarities with idiopathic environmental intolerance (IEI). The nasal biopsy transcriptome of 88 study subjects was profiled using samples obtained at baseline."
!Series_overall_design	"Nasal biopsy samples were collected from female adult-onset asthma patients (n=45), IEI patients (n=14) and healthy subjects (n=21) yielding 80 study subjects. Biopsies were homogenized and total RNA extracted for microarray analyses."
Sample Characteristics Dictionary:
{0: ['diagnosis: healthy', 'diagnosis: adult-onset asthma', 'diagnosis: IEI'], 1: ['gender: Female'], 2: ['age: 38.33', 'age: 38.08', 'age: 48.83', 'age: 33.42', 'age: 46.08', 'age: 45.58', 'ag

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on Series_title and Sample Characteristics, this appears to be transcriptomic profiling data
# This is likely to contain gene expression data, not just miRNA or methylation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (asthma):
# Key 0 contains diagnosis information - healthy, adult-onset asthma, or IEI
trait_row = 0

# For age:
# Key 2 contains age information with multiple unique values
age_row = 2

# For gender:
# Key 1 shows only "gender: Female" - this is a constant feature with only one value
# Since all subjects are female, this is not useful for association analysis
gender_row = None  # Only one gender value (all Female)

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert diagnosis value to binary trait (Asthma: 1, Not Asthma: 0)"""
    if value is None:
        return None
    if ':' in value:
        value = value.split(': ')[1].strip().lower()
    
    if value == 'adult-onset asthma':
        return 1  # Has asthma
    elif value == 'healthy' or value == 'iei':  # IEI (Idiopathic Environmental Intolerance) is not asthma
        return 0  # Does not have asthma
    else:
        return None  # Unknown

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if value is None:
        return None
    if ':' in value:
        value = value.split(': ')[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (Female: 0, Male: 1)"""
    if value is None:
        return None
    if ':' in value:
        value = value.split(': ')[1].strip().lower()
    
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

# 3. Save Metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Note: Skipping clinical feature extraction as we don't have the properly structured clinical data yet.
# The relevant variables (trait_row, age_row, gender_row) and conversion functions 
# (convert_trait, convert_age, convert_gender) have been identified for future steps.
print("Identified variables and conversion functions for future clinical data processing:")
print(f"trait_row = {trait_row}, convert_trait function defined")
print(f"age_row = {age_row}, convert_age function defined")
print(f"gender_row = {gender_row}, convert_gender function defined")


Identified variables and conversion functions for future clinical data processing:
trait_row = 0, convert_trait function defined
age_row = 2, convert_age function defined
gender_row = None, convert_gender function defined


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE182797/GSE182797_series_matrix.txt.gz


Gene data shape: (37616, 80)
First 20 gene/probe identifiers:
Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502',
       'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315529', 'A_19_P00315551',
       'A_19_P00315581', 'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603',
       'A_19_P00315627', 'A_19_P00315631', 'A_19_P00315641', 'A_19_P00315647',
       'A_19_P00315649', 'A_19_P00315668', 'A_19_P00315691', 'A_19_P00315705'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers observed in the data, these appear to be Agilent microarray 
# probe IDs (starting with A_19_P), not standard human gene symbols.
# These identifiers will need to be mapped to official gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation data correspond to:
# - Probe identifiers (same format as in gene expression data)
# - Gene symbols

# The gene expression data uses identifiers starting with "A_19_P" format
# In the gene annotation data, the "ID" column holds these probe identifiers 
# The "GENE_SYMBOL" column holds the gene symbols

# 2. Create the gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Check how many probe IDs have gene symbol mappings
print(f"Total probes with gene symbol mappings: {len(gene_mapping)}")
print("First 5 gene mapping records:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene-level expression using the mapping
# This handles the many-to-many mapping between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Total probes with gene symbol mappings: 48862
First 5 gene mapping records:
               ID    Gene
3   A_33_P3396872   CPED1
4   A_33_P3267760    BCOR
5    A_32_P194264   CHAC2
6    A_23_P153745   IFI30
10  A_21_P0014180  GPR146


Gene expression data shape after mapping: (21476, 80)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF-3', 'A2M', 'A2M-1', 'A2M-AS1', 'A2ML1',
       'A2MP1', 'A4GALT', 'AAAS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Define the correct convert_trait function as established in Step 2
def convert_trait(value: str) -> Optional[int]:
    """Convert trait values to binary (0 for control, 1 for Asthma)."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if "adult-onset asthma" in value.lower():
        return 1  # Asthma
    elif "healthy" in value.lower():
        return 0  # Control
    else:
        return None  # IEI or other conditions

def convert_age(value: str) -> Optional[float]:
    """Convert age values to float."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# Re-extract clinical features using the appropriate conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,  # Correct trait row from Step 2
    convert_trait=convert_trait,
    age_row=2,     # Age row from Step 2
    convert_age=convert_age,
    gender_row=1,  # Gender row from Step 2
    convert_gender=convert_gender
)

# Save the processed clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from adult patients with asthma related to damp/moldy buildings and controls."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Normalized gene data saved to ../../output/preprocess/Asthma/gene_data/GSE182797.csv
Clinical data saved to ../../output/preprocess/Asthma/clinical_data/GSE182797.csv
Linked data shape: (80, 17832)
Linked data preview (first 5 rows, 5 columns):
            Asthma    Age  Gender     A1BG  A1BG-AS1
GSM5537157     0.0  38.33     0.0  7.77916   5.86818
GSM5537158     0.0  38.08     0.0  7.59209   5.59018
GSM5537159     0.0  48.83     0.0  7.45290   5.83891
GSM5537160     1.0  33.42     0.0  7.30202   5.70201
GSM5537161     1.0  46.08     0.0  7.39065   5.76369


Data shape after handling missing values: (66, 17832)
For the feature 'Asthma', the least common label is '0.0' with 21 occurrences. This represents 31.82% of the dataset.
The distribution of the feature 'Asthma' in this dataset is fine.

Quartiles for 'Age':
  25%: 39.582499999999996
  50% (Median): 47.08
  75%: 53.3725
Min: 18.42
Max: 64.67
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 66 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



A new JSON file was created at: ../../output/preprocess/Asthma/cohort_info.json


Linked data saved to ../../output/preprocess/Asthma/GSE182797.csv
